## Setup the local stack (for local testing)

In [ ]:
import os
from loguru import logger

import syft_runtimes.high_low as syhl
from syft_rds.orchestra import setup_rds_server, remove_rds_stack_dir
from syft_core.config import CONFIG_PATH_ENV
import syft_datasets as syd

In [ ]:
remove_rds_stack_dir(key="low_datasites", root_dir="./")
!rm -rf ./high_datasites

In [ ]:
DO = "do1@openmined.org"
do_stack_1 = setup_rds_server(email=DO, key="low_datasites", root_dir="./")

In [ ]:
lowside_client = do_stack_1.client
lowside_syftbox_dir = do_stack_1.client.workspace.data_dir
highside_syftbox_dir = lowside_syftbox_dir.parent / "high_datasites"
lowside_email = DO

In [ ]:
lowside_client.email

## DO (High): Initializing the high datasite

In [ ]:
# Low side settings
print(f"{lowside_syftbox_dir = }")
print(f"{highside_syftbox_dir = }")

In [ ]:
highlow_identifier = "highlow-1234"

# First, initialize the high datasite - runs on the low side
highside_client: syhl.HighSideClient = syhl.initialize_high_datasite(
    email=DO,
    highlow_identifier=highlow_identifier,
    syftbox_dir=highside_syftbox_dir,
    lowside_syftbox_dir=lowside_syftbox_dir,
    connection_config=None,  # default to local lowside connection
    force_overwrite=True,
)

In [ ]:
highside_client.workspace.data_dir

## DO (high): Create a high-side dataset

In [ ]:
# Create some random mock and private data
from pathlib import Path
import random
import pandas as pd

num_mock_rows = 100
num_private_rows = 100

mock_data = {
    "age": [random.randint(0, 100) for _ in range(num_mock_rows)],
    "height": [random.uniform(150, 200) for _ in range(num_mock_rows)],
    "income": [random.randint(20000, 100000) for _ in range(num_mock_rows)],
}
private_data = {
    "age": [random.randint(0, 100) for _ in range(num_private_rows)],
    "height": [random.uniform(150, 200) for _ in range(num_private_rows)],
    "income": [random.randint(20000, 100000) for _ in range(num_private_rows)],
}

mock_df = pd.DataFrame(mock_data)
private_df = pd.DataFrame(private_data)

data_dir = Path("./data")
data_dir.mkdir(parents=True, exist_ok=True)
mock_df.to_csv(data_dir / "mock_data.csv", index=False)
private_df.to_csv(data_dir / "private_data.csv", index=False)

readme_content = """
# My example high-side dataset
"""
readme_path = data_dir / "README.md"
readme_path.write_text(readme_content)

In [ ]:
os.environ[CONFIG_PATH_ENV] = str(highside_client.config_path)

DATASET_NAME = "my_cool_dataset"

highside_dataset = syd.create(
    name=DATASET_NAME,
    mock_path=data_dir / "mock_data.csv",
    private_path=data_dir / "private_data.csv",
    readme_path=readme_path,
)

In [ ]:
highside_dataset

In [ ]:
highside_dataset.mock_dir

In [ ]:
highside_dataset.private_dir

In [ ]:
# We can now see the dataset on the low side!
from syft_datasets import SyftDatasetManager

highside_dataset_manager = SyftDatasetManager(syftbox_client=highside_client)
highside_datasets = highside_dataset_manager.get_all()

highside_datasets

## DO (High): Rsync the dataset (mock part only) to the low side

In [ ]:
# Sync the dataset (mock part) to the low side
highside_client.sync_dataset(
    dataset_name=DATASET_NAME,
    verbose=True,
)

## DO (High): Rsync another dataset (mock part only) to the low side

In [ ]:
os.environ[CONFIG_PATH_ENV] = str(highside_client.config_path)

DATASET_2_NAME = "my_cool_dataset_2"

highside_dataset = syd.create(
    name=DATASET_2_NAME,
    mock_path=data_dir / "mock_data.csv",
    private_path=data_dir / "private_data.csv",
    readme_path=readme_path,
)

In [ ]:
# Sync the dataset (mock part) to the low side
highside_client.sync_dataset(
    dataset_name=DATASET_2_NAME,
    verbose=True,
)

## DO (Low): Checks if the dataset exists (only the mock part)

In [ ]:
# We can now see the dataset on the low side!
from syft_datasets import SyftDatasetManager

lowside_syftbox_client = do_stack_1.client

lowside_dataset_manager = SyftDatasetManager(syftbox_client=lowside_syftbox_client)
lowside_datasets = lowside_dataset_manager.get_all()

lowside_datasets

In [ ]:
for dataset in lowside_datasets:
    print(dataset.mock_dir)

In [ ]:
for dataset in lowside_datasets:
    try:
        print(dataset.private_dir)
    except Exception as e:
        logger.error(
            f"Can't access private dataset from low side because it's a high-low dataset: {e}"
        )

## DS: sees that the mock dataset is uploaded to the SyftBox network, play with it and submits a job to the DO's low datasite
```python
from syft_rds import init_session

ds_client = init_session(host=DO)

ds_client.job.submit(
    name="job_name",
    user_code_path="path_to_code",
    dataset_name="dataset_name_to_run_on",
    entrypoint="main.py",
)
```

The job will go to the public folder of the DO

## DO (Low): Review and approve jobs

```python
jobs = lowside_syftbox_client.job.get_all(status="pending_code_review")

lowside_syftbox_client.approve(
    jobs[0]
)  # move the job to private/<email>/syft_runtimes/<runtime_name>/jobs
```

In [ ]:
# mock DS submission and DO approval

print(f"{highside_client._get_lowside_runtime_dir() = }")
(highside_client._get_lowside_runtime_dir() / "jobs" / "job01").mkdir(
    exist_ok=True, parents=True
)
(highside_client._get_lowside_runtime_dir() / "jobs" / "job01" / "main.py").touch()

## DO (High): Rsync pending jobs from the low side client

In [ ]:
highside_client.sync_pending_jobs()

assert (highside_client.runtime_dir / "jobs" / "job01").exists()
assert (highside_client.runtime_dir / "jobs" / "job01" / "main.py").exists()

## DO (High): Runs the job on private data

```python
pending_jobs = highside_client.job.get_all(status="pending_code_review")  # get all pending jobs
for job in pending_jobs:
    highside_client.run_private(job)  # run the job using syft_runtimes
```

In [ ]:
# mock DO runs job successfully

print(f"{highside_client.runtime_dir = }")
(highside_client.runtime_dir / "done" / "job01").mkdir(exist_ok=True, parents=True)
(highside_client.runtime_dir / "done" / "job01" / "output.txt").touch()

## DO (High): Review and rsync job results in the `done` folder to the lowside client

In [ ]:
highside_client.sync_done_jobs()

In [ ]:
assert (highside_client._get_lowside_runtime_dir() / "done" / "job01").exists()
(highside_client._get_lowside_runtime_dir() / "done" / "job01" / "output.txt").exists()